# Part 1

In [ ]:
! pip install chembl_webresource_client

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client


## Target Search


In [ ]:
# Target search for coronavirus
target = new_client.target
target_query = target.search('acetylcholinesterase')
targets = pd.DataFrame.from_dict(target_query)
targets


In [ ]:

selected_target = targets.target_chembl_id[0]
selected_target

In [ ]:


activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [97]:
df = pd.DataFrame.from_dict(res)
df


KeyboardInterrupt: 

In [ ]:

df.to_csv('bioactivity_data_raw.csv', index=False)


In [ ]:

df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

In [ ]:

len(df2.canonical_smiles.unique())

df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr
     

In [ ]:

selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3

In [ ]:

df3.to_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv', index=False)

In [ ]:
df4 = pd.read_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv')

In [ ]:

bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [ ]:

bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5
     

In [ ]:
df5.to_csv('acetylcholinesterase_03_bioactivity_data_curated.csv', index=False)

# Part 2

### Install Conda and RdKit

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
df = pd.read_csv('acetylcholinesterase_03_bioactivity_data_curated.csv')
df

In [ ]:
df_no_smiles = df.drop(columns='canonical_smiles')

In [ ]:
smiles = []

for i in df.canonical_smiles.tolist():
  cpd = str(i).split('.')
  cpd_longest = max(cpd, key = len)
  smiles.append(cpd_longest)

smiles = pd.Series(smiles, name = 'canonical_smiles')

In [ ]:
df_clean_smiles = pd.concat([df_no_smiles,smiles], axis=1)
df_clean_smiles

In [ ]:
import numpy as np
! pip install rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

### Calculate Lipinski Descriptors

In [ ]:
# Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [ ]:

df_lipinski = lipinski(df_clean_smiles.canonical_smiles)
df_lipinski

### Combine Dataframe

In [ ]:
df_combined = pd.concat([df,df_lipinski], axis=1)

In [ ]:
df_combined

### Convert IC50 to pIC50

In [ ]:
df_combined.standard_value.describe()

In [ ]:
def norm_value(input):
    norm = []
    for i in input['standard_value']:
        if i > 100000000:
            i = 100000000
        norm.append(i)
    input['standard_value_norm'] = norm
    return input.drop('standard_value', axis=1)

def pIC50(input):
    pIC50 = []
    for i in input['standard_value_norm']:
        molar = i*(10**-9)  # Convert nM to M
        pIC50.append(-np.log10(molar))
    input['pIC50'] = pIC50
    return input.drop('standard_value_norm', axis=1)

df_norm = norm_value(df_combined)
df_final = pIC50(df_norm)

In [ ]:
df_final.pIC50.describe()


### Removing the 'intermediate' bioactivity class

In [ ]:
df_2class = df_final[df_final.bioactivity_class != 'intermediate']
df_2class

### EDA

In [ ]:
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt


In [ ]:
# Set Seaborn style
sns.set_style("whitegrid")
sns.set_context("talk")  # This makes elements of the plot bigger, good for presentations

# Create a color palette
palette = sns.color_palette("pastel")

# Create the plot
plt.figure(figsize=(8, 6))
ax = sns.countplot(x='bioactivity_class', data=df_2class, palette=['mediumpurple','teal'], edgecolor='black')

# Set title and labels
ax.set_title('Distribution of Bioactivity Classes', fontsize=16, fontweight='bold')
ax.set_xlabel('Bioactivity class', fontsize=14, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=14, fontweight='bold')

# Annotate bar plots with their respective values
for p in ax.patches:
    ax.annotate(f'{p.get_height()}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                xytext=(0, 10),
                textcoords='offset points',
                fontsize=12)

# Reduce visibility of top and right spines
sns.despine(right=True, top=True)

# Save the plot
plt.tight_layout()  # Ensure that all elements fit well and are not cut off in the saved figure
plt.savefig('plot_bioactivity_class.pdf')
plt.show()

### Scatter plot of MW vs LogP

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.scatterplot(x='MW', y='LogP', data=df_2class, hue='bioactivity_class', size='pIC50', edgecolor='black', alpha=0.7, palette=['mediumpurple','teal'])

plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig('plot_MW_vs_LogP.pdf')
plt.show()


### pIC50 Value

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x='bioactivity_class', y='pIC50', data=df_2class, palette=['mediumpurple','teal'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel(r'pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_ic50.pdf')
plt.show()


### Mann-Whitney U Test

In [ ]:
def mannwhitney(descriptor, verbose=False):
  # https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
  from numpy.random import seed
  from numpy.random import randn
  from scipy.stats import mannwhitneyu

# seed the random number generator
  seed(1)

# actives and inactives
  selection = [descriptor, 'bioactivity_class']
  df = df_2class[selection]
  active = df[df.bioactivity_class == 'active']
  active = active[descriptor]

  selection = [descriptor, 'bioactivity_class']
  df = df_2class[selection]
  inactive = df[df.bioactivity_class == 'inactive']
  inactive = inactive[descriptor]

# compare samples
  stat, p = mannwhitneyu(active, inactive)
  #print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
  alpha = 0.05
  if p > alpha:
    interpretation = 'Same distribution (fail to reject H0)'
  else:
    interpretation = 'Different distribution (reject H0)'

  results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
  filename = 'mannwhitneyu_' + descriptor + '.csv'
  results.to_csv(filename)

  return results


In [ ]:
mannwhitney('pIC50')


In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x='bioactivity_class', y='MW', data=df_2class, palette=['mediumpurple','teal'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel(r'pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.pdf')
plt.show()


In [ ]:
mannwhitney('MW')



In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x='bioactivity_class', y='LogP', data=df_2class, palette=['mediumpurple','teal'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel(r'pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_LogP.pdf')
plt.show()


In [ ]:
mannwhitney('LogP')



In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x='bioactivity_class', y='NumHDonors', data=df_2class, palette=['mediumpurple','teal'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel(r'pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHDonors.pdf')
plt.show()


In [ ]:
mannwhitney('NumHDonors')




In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x='bioactivity_class', y='NumHAcceptors', data=df_2class, palette=['mediumpurple','teal'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel(r'pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHAcceptors.pdf')
plt.show()


In [ ]:
mannwhitney('NumHAcceptors')




Zip Function

In [ ]:
# ! zip -r results.zip . -i *.csv *.pdf

### Part 3